This notebook provides a summary of our approach to the project, with any issues we face and limitations/assumptions made.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

data_directory = "../data/curated/"
merchants = pd.read_parquet(data_directory + "merchants.parquet")
transactions = pd.read_parquet(data_directory + "transactions.parquet")
consumers = pd.read_parquet(data_directory + "consumers.parquet")
census = pd.read_csv(data_directory + "census.csv")
#merged_transactions = pd.read_parquet(data_directory + "merged_transactions.parquet")

FileNotFoundError: [Errno 2] No such file or directory: '../data/curated/transactions.parquet'

# Normalisation

Firstly, we noticed an issue with the merchant dataset: the 'tags' column was incorrectly constructed with an overload of information - it required normalisation, not only because it showcases technical competency, but the separation could be useful in ranking later on (for example, take-rate, revenue band and category of purchase may all be relevant when ranking and, ideally, should be in different columns).

Hence, a normalisation procedure was conducted on this column and the corresponding ETL implementation was done, too. 

In [8]:
merchants.head()

,name,tags,sector_tags,revenue_band,take_rate
merchant_abn,,,,,
10023283211,Felis Limited,"((furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",e,0.18
10142254217,Arcu Ac Orci Corporation,"([cable, satellite, and otHer pay television a...","cable, satellite, and other pay television and...",b,4.22
10165489824,Nunc Sed Company,"([jewelry, watch, clock, and silverware shops]...","jewelry, watch, clock, and silverware shops",b,4.40
10187291046,Ultricies Dignissim Lacus Foundation,"([wAtch, clock, and jewelry repair shops], [b]...","watch, clock, and jewelry repair shops",b,3.29
10192359162,Enim Condimentum PC,"([music shops - musical instruments, pianos, a...","music shops - musical instruments, pianos, and...",a,6.33


# Preprocessing/Outlier Analysis

After downloading/extracting the relevant datasets that were provided to us and conducting some preliminary analysis, the first step was to examine outliers and determine a method to eliminate them from the dataset. 

We found that, after merging transactions with merchants, the nulls are predominantly related to the name of the merchant and what they purchased ('tags'). At the preliminary stage, it was determined that these can be considered vital features when ranking merchants, and if we lack this data, we hinder the accuracy of our selection metrics. Hence, all nulls were removed from further analysis.

To conduct further outlier analysis, we utilised a box plot (per merchant) to visualise the anomalies:

In [ ]:
transactions_noNull = transactions.merge(merchants, on='merchant_abn').dropna().groupby("merchant_abn").agg(outlier_count=("dollar_value", get_outliers))

def get_outliers(col):
    q1 = np.percentile(column, 25)
    q3 = np.percentile(column, 75) 
    IQR = q3 - q1
    return sum((column<(q1 - IQR)) | (column>(q3 + IQR)))
plt.figure(figsize=(10,10))
#sns.boxplot(x='name', y='dollar_value', data = transactions_noNull)


#### INSERT BOX-PLOT/VISUALISATION OF OUTLIERS HERE ####

To eliminate the observed outliers, an IQR statistical implementation was used, resulting in approximately 3% of all data (113982 rows) being removed.

Complementing all these steps, a generic ETL script was being updated using separate functions for obtaining and preprocessing the data.

### Using a Fraud Model to Further Remove Data

After the release of the fraud dataset, we wanted to implement a model that detects fraudulent transactions and also removes them from the dataset. As the fraud data is only a subset of transactions deemed suspicious, we will deal with outliers that represent those transactions deemed suspicious based on our analysis, instead of the entire dataset.

In order to identify outliers, we had to normalise the dollar values according to each customer using Median and Quantile Scaling, and then train a linear regression model on the transaction-fraud data. Then, we would apply this model to the outliers (dollar values >$2 after the scaling) in the rest of the dataset. In the end, we used the predicted probability in conjunction with a randomly generated probability to remove the fraud data (for example, if probability of fraud is 60%, then there is a 60% chance it will be removed from the dataset).

### External Dataset

The external dataset (census data) was retrieved from the ABS website. We believed that some of these features could prove useful in deriving a ranking model. Additionally, as the census data linked customer data by postcode and gender, it is assumed that this average is representative of the individual.

Due to the hundreds of features available, feature engineering was done in order to obtain the most predictive attributes of a consumer. For example, a house-repayment-to-income ratio was deemed important as it showcases how risky - or how likely to default - a consumer is, and was engineered by dividing median mortgage payment of a consumer by their median income.

The data was then cleaned and merged into other datasets.

 ### Final (Merged) Dataset

By the preprocessing and outlier analysis detailed above for each dataset, we were able to obtain a final dataset by merging.

The ETL script was finalised, also. 

In [26]:
#merged_transactions = pd.read_parquet(data_directory + "merged_transactions.parquet")


####INCLUDE FINAL, MERGED DATASET HERE####

TypeError: unsupported operand type(s) for +: 'int' and 'dict'

# The Ranking Model and Segmentations

Initially, we devised a metric using three models: Customer-Merchant Model (CN), Customer Number Model (CNM) and Customer Sampling Model (CS). The CN model would be used to predict the amount each cutomer-merchant pair would spend on a monthly basis; the CNM model in conjunction with CS would be used to predict revenue for a specific month (using a Monte-Carlo sampling method). Ultimately, we would use all of this information to predict the future revenue for any given month for a specific merchant. However, we faced a number of issues in creation of this system:
- Customer were sparse: we found that each customer only had a few transactions per merchant, hence making it difficult to derive a predictive power
- Memory/Technical Issues: since these databases/operations were computationally heavy, we were met with several RAM issues across group members, as well as .env and environment errors which impeded the creation process significantly 

As a result, we decided to change the model to the one we have now. By using several, reputable articles and studies, we were able to select and create features:
- Prevalent in Afterpay's annual report was the notion that the majority of revenue comes from the take-rate - hence, highlighting the fact that this should be the most important aspect of our model
- Customer Retention served as a distinguishing factor amongst the top chosen merchants - some merchants with comparitively lower revenue scores were still pinpointed by our model due to the comparitively higher retention score, which is an interesting discovery
- Research also revealed characteristics of BNPL customers which was accounted for in our Customer Quality metric:
    - Federal Reserve Bank of Philadelphia stated that the BNPL market is far more attractive to the younger audience
    - Federal Reserve Bank of Philadelphia also advised us to avoid nil income customers as they are likely to default
    - Seek low mortgage-income ratio, as mortgage stress is a strong indicator of financial position
    - Roy Morgan (reputable market research company) stated avoid high income customers as they are unlikely to use a BNPL approach

This research prompted us to weight the features accordingly. In regard to the Customer Quality metric, we found it interesting how amongst the top merchants, the mean quality of customers converges to the mean of the individual customer quality distribution. The implication is that each merchant's customers are chosen randomly from the overall customer distribution. Possibly with more realistic data, this would be more useful. 

(INSERT A VISUAL THAT SHOWCASES THE CONVERGING CUSTOMER QUALITY STUFF)

Below is our final ranking of merchants: 

In [7]:
import pandas as pd
rankings = pd.read_csv('../results/rankings.csv')
rankings.head(10)

,merchant_abn,name,sector_tags,retention,momentum_score,revenue_score,customer_quality,score
0,86578477987,Leo In Consulting,"watch, clock, and jewelry repair shops",0.969028,0.580496,0.952110,0.770702,0.880765
1,32361057556,Orci In Consequat Corporation,"gift, card, novelty, and souvenir shops",0.664392,0.616855,1.000000,0.774540,0.869641
2,64403598239,Lobortis Ultrices Company,"music shops - musical instruments, pianos, and...",0.773248,0.602984,0.965683,0.764959,0.862341
3,89726005175,Est Nunc Consulting,tent and awning shops,0.937524,0.613811,0.861313,0.766881,0.826176
4,21439773999,Mauris Non Institute,"cable, satellite, and other pay television and...",0.780963,0.595851,0.894282,0.777622,0.820854
5,45629217853,Lacus Consulting,"gift, card, novelty, and souvenir shops",0.941055,0.575384,0.819798,0.763485,0.795693
6,48534649627,Dignissim Maecenas Foundation,"opticians, optical goods, and eyeglasses",0.560206,0.614034,0.858046,0.763377,0.767301
7,94493496784,Dictum Phasellus In Institute,"gift, card, novelty, and souvenir shops",0.717041,0.584147,0.815619,0.769235,0.761473
8,49322182190,Gravida Mauris Incorporated,"watch, clock, and jewelry repair shops",0.492738,0.578918,0.862436,0.762769,0.754487
9,96680767841,Ornare Limited,motor vehicle supplies and new parts,0.328149,0.564826,0.897430,0.780714,0.750476


### Segmentations

Our segmentations are based on a report by Citibank which highlighted different Merchant Category Codes and provided descriptions for each one. 

For reference: https://www.citibank.com/tts/solutions/commercial-cards/assets/docs/govt/Merchant-Category-Codes.pdf

We simply viewed all the sector tags in our dataset and placed them under the specifc merchant category by matching descriptions from the report.

Below are our segments:

In [19]:
business = pd.read_csv('../results/Business Services_rankings.csv')
misc = pd.read_csv('../results/Miscellaneous Stores_rankings.csv')
prof = pd.read_csv('../results/Professional services and membership organisations_rankings.csv')
retail = pd.read_csv('../results/Retail outlet services_rankings.csv')
utility = pd.read_csv('../results/Utility services_rankings.csv')

In [14]:
#Business Services Segment Ranking
business.head()

,merchant_abn,name,sector_tags,retention,momentum_score,revenue_score,customer_quality,score
0,35909341340,Arcu Sed Eu Incorporated,"computer programming , data processing, and in...",0.377500,0.597597,0.708582,0.744300,0.645844
1,58454491168,Diam At Foundation,"computer programming , data processing, and in...",0.279798,0.483744,0.617034,0.767027,0.561454
2,67400260923,Eleifend PC,"computer programming , data processing, and in...",0.381212,0.562784,0.502232,0.764662,0.519405
3,77590625261,Sed Diam Foundation,"computer programming , data processing, and in...",0.272368,0.632808,0.387516,0.761563,0.444442
4,94455880010,Nunc Sit LLC,"computer programming , data processing, and in...",0.287177,0.541501,0.231515,0.766321,0.339843


In [15]:
#Miscellaneous Stores Segment Ranking
misc.head()

,merchant_abn,name,sector_tags,retention,momentum_score,revenue_score,customer_quality,score
0,86578477987,Leo In Consulting,"watch, clock, and jewelry repair shops",0.969028,0.580496,0.952110,0.770702,0.880765
1,32361057556,Orci In Consequat Corporation,"gift, card, novelty, and souvenir shops",0.664392,0.616855,1.000000,0.774540,0.869641
2,64403598239,Lobortis Ultrices Company,"music shops - musical instruments, pianos, and...",0.773248,0.602984,0.965683,0.764959,0.862341
3,89726005175,Est Nunc Consulting,tent and awning shops,0.937524,0.613811,0.861313,0.766881,0.826176
4,45629217853,Lacus Consulting,"gift, card, novelty, and souvenir shops",0.941055,0.575384,0.819798,0.763485,0.795693


In [16]:
#Professional Services Segment Ranking
prof.head()

,merchant_abn,name,sector_tags,retention,momentum_score,revenue_score,customer_quality,score
0,48534649627,Dignissim Maecenas Foundation,"opticians, optical goods, and eyeglasses",0.560206,0.614034,0.858046,0.763377,0.767301
1,46804135891,Suspendisse Dui Corporation,"opticians, optical goods, and eyeglasses",0.944943,0.580593,0.288396,0.764694,0.478337
2,95574756848,At Pede Inc.,"opticians, optical goods, and eyeglasses",0.133073,0.569563,0.373195,0.771369,0.406450
3,92065881715,Quam Elementum Corp.,"opticians, optical goods, and eyeglasses",0.107265,0.613529,0.190654,0.765458,0.299057
4,11237511112,Magna Institute,"opticians, optical goods, and eyeglasses",0.225500,0.605133,0.137054,0.764276,0.283255


In [17]:
#Retail Outlet Services Segment Ranking
retail.head()

,merchant_abn,name,sector_tags,retention,momentum_score,revenue_score,customer_quality,score
0,96680767841,Ornare Limited,motor vehicle supplies and new parts,0.328149,0.564826,0.897430,0.780714,0.750476
1,43186523025,Lorem Ipsum Sodales Industries,"florists supplies, nursery stock, and flowers",0.922267,0.581939,0.650190,0.766660,0.692411
2,45433476494,Adipiscing Elit Foundation,"computers, computer peripheral equipment, and ...",0.187506,0.558081,0.685691,0.738824,0.597135
3,24852446429,Erat Vitae LLP,"florists supplies, nursery stock, and flowers",0.974489,0.586088,0.398765,0.766150,0.549961
4,49212265466,Auctor Company,"florists supplies, nursery stock, and flowers",0.366647,0.571828,0.538097,0.769335,0.540563


In [18]:
#Utility Services Segment Ranking
utility.head()

,merchant_abn,name,sector_tags,retention,momentum_score,revenue_score,customer_quality,score
0,21439773999,Mauris Non Institute,"cable, satellite, and other pay television and...",0.780963,0.595851,0.894282,0.777622,0.820854
1,82368304209,Nec Incorporated,telecom,0.060253,0.723983,0.700684,0.767242,0.614770
2,17488304283,Posuere Cubilia Curae Corporation,"cable, satellite, and other pay television and...",0.353629,0.577238,0.326002,0.761587,0.411390
3,93260930990,Pede Cras Vulputate Ltd,telecom,0.022406,0.498862,0.375875,0.763826,0.380098
4,68004106739,Nec Ante Ltd,"cable, satellite, and other pay television and...",0.217687,0.610719,0.132751,0.762972,0.280209


### Interesting Merchants

After producing the rankings, we noticed some interesting merchants. When we ordered by score in our rankings dataset, one merchant particularly stood out. Despite being in the top 10 merchants, 'Dictum Phasellus In Institute' has the lowest revenue amongst this group. However, the retention score is comparitively high, further supporting the claim that retention is a powerful separator between merchants. 

Additionally, 'Orci In Consequat Coorperation' is a top 2 merchant, despite having one of the lowest retention scores and average customer quality score. But, this merchant was scored a 1.00 in regards to revenue, reinforcing the importance of this factor in the model.